In [1]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from numba import autojit
from collections import Counter
%matplotlib inline
import json

In [2]:
@autojit
def two_point_correlation(im, dim):
    if dim == 0: #x_direction
        dim_1 = im.shape[2] #y-axis
        dim_2 = im.shape[1] #z-axis
        dim_3 = im.shape[0] #x-axis
    elif dim == 1: #y-direction
        dim_1 = im.shape[0] #x-axis
        dim_2 = im.shape[1] #z-axis
        dim_3 = im.shape[2] #y-axis
    elif dim == 2: #z-direction
        dim_1 = im.shape[0] #x-axis
        dim_2 = im.shape[2] #y-axis
        dim_3 = im.shape[1] #z-axis
        
    two_point = np.zeros((dim_1, dim_2, dim_3))
    for n1 in range(dim_1):
        for n2 in range(dim_2):
            for r in range(dim_3):
                lmax = dim_3-r
                for a in range(lmax):
                    if dim == 0:
                        pixel1 = im[a, n2, n1]
                        pixel2 = im[a+r, n2, n1]
                    elif dim == 1:
                        pixel1 = im[n1, n2, a]
                        pixel2 = im[n1, n2, a+r]
                    elif dim == 2:
                        pixel1 = im[n1, a, n2]
                        pixel2 = im[n1, a+r, n2]
                    
                    if pixel1 == 0 and pixel2 == 0:
                        two_point[n1, n2, r] += 1
                two_point[n1, n2, r] = two_point[n1, n2, r]/(float(lmax))
    return two_point

In [3]:
names = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "C1", "C2", "Berea"]

for name in names:
    print name
    im_crop = tifffile.imread("image_data/images/"+name+".tif")
    two_point_x = two_point_correlation(im_crop, 1)
    two_point_y = two_point_correlation(im_crop, 2)
    two_point_z = two_point_correlation(im_crop, 0)
    print "computed covariances"
    mean_two_point_x = np.mean(np.mean(two_point_x, axis=0), axis=0)
    mean_two_point_y = np.mean(np.mean(two_point_y, axis=0), axis=0)
    mean_two_point_z = np.mean(np.mean(two_point_z, axis=0), axis=0)
    mean_two_point = np.mean(np.vstack([mean_two_point_x, mean_two_point_y, mean_two_point_z]), axis=0)
    poro_data = Counter(im_crop.flatten())
    porosity = (poro_data[0]/float(poro_data[0]+poro_data[255]))
    print "computed porosity"
    two_point_x_l = list(mean_two_point_x)
    two_point_y_l = list(mean_two_point_y)
    two_point_z_l = list(mean_two_point_z)
    two_point_l = list(mean_two_point)

    two_point_data = {"name":name, 
                      "x": two_point_x_l, 
                      "y": two_point_y_l, 
                      "z": two_point_z_l, 
                      "porosity": porosity, 
                      "averaged": two_point_l}
    with open("covariance_results/"+name+".json", 'w') as outfile:
        json.dump(two_point_data, outfile)
    print "output to file"

S1
computed covariances
computed porosity
output to file
S2
computed covariances
computed porosity
output to file
S3
computed covariances
computed porosity
output to file
S4
computed covariances
computed porosity
output to file
S5
computed covariances
computed porosity
output to file
S6
computed covariances
computed porosity
output to file
S7
computed covariances
computed porosity
output to file
S8
computed covariances
computed porosity
output to file
S9
computed covariances
computed porosity
output to file
C1
computed covariances
computed porosity
output to file
C2
computed covariances
computed porosity
output to file
Berea
computed covariances
computed porosity
output to file
